In [20]:
import requests
import xmltodict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import torch
import time

In [21]:
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 4060
Using device: cuda


In [22]:
import ollama

# Define the system prompt that sets the behavior
system_prompt = """
You are an AI assistant tasked with generating short specific and high-level questions based on a given abstract. Your responses must always follow the rules and format below:

### Rules:
1. Generate **exactly 5 questions** based on the provided abstract.
2. Structure the questions as follows:
   - **4 very generic, short, high-level questions** that cover foundational knowledge or broad concepts related to the abstract. These questions should:
     - Address general definitions, underlying concepts, or methodologies.
     - Be broadly applicable to related topics, even beyond the specific focus of the abstract.
     - Be something that a high school or college student can ask about. 
   - **1 specific, detailed questions** that focus on the methods, results, or technical contributions explicitly mentioned in the abstract.
3. Do **not refer to the abstract itself** or use phrases like "this paper," "this abstract," or "this research."
4. Ensure the questions are self-contained and clear, making sense without the abstract for context.
5. Avoid vague questions like "What is the significance of this research?" Instead, focus on relevance to the abstract's content or broader domain.
6. Don't include any additional phrases in your responses outside of the list of questions. 
7. Each question should be a short and simple sentence. 

### Format:
Return the questions as a numbered list, where each question starts with "Question X:" (e.g., "Question 1:").
For example:
   Question 1: <Generic Question>
   Question 2: <Generic Question>
   Question 3: <Generic Question>
   Question 4: <Generic Question>
   Question 5: <Specific Question>

### Example Abstract (Placeholder Text):
"The ClimateBERT model is a transformer-based architecture fine-tuned on climate-related datasets. It aims to capture public sentiment about climate change through advanced language modeling techniques, providing unique insights into societal responses to climate issues."

### Example Output:
Question 1: What is ClimateBERT, and how is it different from other pre-trained language models?  
Question 2: What are transformers, and why are they important to ClimateBERT or BERT-based architectures?  
Question 3: How can a generic BERT model be fine-tuned to domain-specific datasets?  
Question 4: What does BERT stand for? 
Question 5: How does ClimateBERT address challenges like bias or noise in sentiment analysis for climate-related data?  
"""

# Function to process a single abstract
def generate_questions_for_abstract(abstract):
    response = ollama.chat(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": f"Abstract: {abstract}",
            },
        ],
    )
    return response["message"]["content"]

In [23]:
df = pd.read_json('papers.json', orient='records', lines=True)
len(df)

2664

In [24]:
from tqdm import tqdm

abstracts = df['summary'].tolist()

# Process each abstract and collect results
results = []
for abstract in tqdm(abstracts, desc="Processing Abstracts"):
    questions = generate_questions_for_abstract(abstract)
    results.append(questions)

Processing Abstracts:   0%|          | 0/2664 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [8]:
df['questions'] = results
df.to_json('papers1.json', orient='records', lines=True)